In [57]:
import numpy as np
import pysocialforce as psf
import os
import math
import cv2
import random
import instances.wide_group_radius100_2ppl as instance

Human simulator initialization.
Definition of Humans state, social groups and obstacles.
Here are dfined different kind of obstacles, some of the m are real obstacles, other are fake and they will be used for labeling, mainly for ques, so a right bound and a left bound obstacles are defined, like in the paper from Andrea, this allows for automatically generating the label for the output.

In [58]:
#do some resoning about the rotation/dilation cases, and hot to instantiate the simulator in this case
def sim_init(instance):
    #modify global variables
    global initial_state, groups, real_obs, queue_obs, scaling_obs, obs, simulation_steps, x_dim_grid, y_dim_grid, robot_start_cont, robot_goal_cont, x_q_min, x_q_max, y_q_min, y_q_max, queue_initial_state, people_in_group, group_radius

    # initial states, each entry is the position, velocity and goal of a pedestrian 
    # in the form of (px, py, vx, vy, gx, gy) Please define only positive positions
    initial_state = instance.initial_state

    people_in_group = instance.people_in_group
    group_radius = instance.group_radius

    # social groups informoation is represented as lists of indices of the state array
    #groups = instance.groups
    groups = [[-1]]

    # list of linear obstacles given in the form of (x_min, x_max, y_min, y_max) Please define only positive coordinates
    real_obs = instance.obs
    queue_obs = instance.queue_obs
    scaling_obs = instance.scaling_obs
    if real_obs.size >0 and queue_obs.size>0:
        print('both non empty')
        obs = np.append(real_obs, [queue_obs, scaling_obs], axis=0)
    elif queue_obs.size >0: 
        obs = np.append(queue_obs, scaling_obs, axis=0)
    elif real_obs.size>0: 
        obs = np.append(real_obs, scaling_obs, axis=0)
    else: obs = scaling_obs

    
    simulation_steps = instance.simulation_steps

    x_dim_grid = instance.dim_grid
    y_dim_grid = x_dim_grid

    robot_start_cont = instance.robot_start_cont
    robot_goal_cont = instance.robot_goal_cont

    '''randomly add humans in the queue if the queue obstacle array is not empty 
        one person (randomly) every one meter segment'''
    if queue_obs.size >0:
        x_q_min = 25
        x_q_max = 0
        y_q_min = 25
        y_q_max = 0
        for obst in queue_obs:
            if obst[0] < x_q_min:
                x_q_min = obst[0]
            if obst[1] > x_q_max:
                x_q_max = obst[1]
            
            if obst[2] < y_q_min:
                y_q_min = obst[2]
            if obst[3] > y_q_max:
                y_q_max = obst[3]


        seg_factor = 1.1
        segments = math.floor((x_q_max-x_q_min)/seg_factor)
        
        queue_initial_state = np.zeros((segments, 6))
        speed_demultiplier = 0.1
        for segment in range(segments):
            queue_initial_state[segment, 0] = random.uniform(x_q_min+seg_factor*segment, x_q_min+seg_factor*segment+1, )
            queue_initial_state[segment, 1] = random.uniform(y_q_min, y_q_max)
            x_speed = speed_demultiplier*(robot_goal_cont[0] - queue_initial_state[segment, 0])/(robot_goal_cont[0] - queue_initial_state[segment, 0])**2
            queue_initial_state[segment, 2] = x_speed + 0.1* random.uniform(-0.1 *x_speed, 0.1*x_speed)
            y_speed = speed_demultiplier*(robot_goal_cont[1] - queue_initial_state[segment, 1])/(robot_goal_cont[1] - queue_initial_state[segment, 1])**2
            queue_initial_state[segment, 3] = y_speed + 0.1 * random.uniform(-0.1 *y_speed, 0.1*y_speed)
            queue_initial_state[segment, 4] = robot_goal_cont[0]
            queue_initial_state[segment, 5] = robot_goal_cont[1]

        

        # append humans
        if initial_state.size >0:
            initial_state = np.concatenate((initial_state, queue_initial_state), axis=0)
        
        
        return obs
                




In [59]:
def shortest_angular_distance(from_angle, to_angle):
    # Compute the difference between the two angles
    diff = to_angle - from_angle

    # Normalize the difference to be within the range of -pi to pi
    while diff < -math.pi:
        diff += 2 * math.pi
    while diff > math.pi:
        diff -= 2 * math.pi

    return diff


place humans on gridmap

In [60]:
def humans_in_gridmap(humans, human_gridmap):
    # Determine the maximum x and y values for scaling
    x_max = np.max(scaling_obs[:, 1])
    y_max = np.max(scaling_obs[:, 3])

    # Scale humans' positions to grid dimensions
    scale_hum2grid = np.diag([x_dim_grid / x_max, y_dim_grid / y_max])
    humans_grid_pos = np.uint16(np.floor(humans[:, :2] @ scale_hum2grid))

    # Update the human_gridmap in place
    for i in range(humans_grid_pos.shape[0]):
        human_gridmap[humans_grid_pos[i, 0], humans_grid_pos[i, 1]] = 100
        dx = 0
        dy = 0
        vx = humans[i, 2]
        vy = humans[i, 3]
        ratio_threshold = 2.0

        # Avoid division by zero
        if vx == 0 and vy == 0:
            print('unknown')
        
        # Calculate absolute values and ratios
        abs_vx = abs(vx)
        abs_vy = abs(vy)
        
        if abs_vx > abs_vy * ratio_threshold:
            dx = np.sign(vx)
        elif abs_vy > abs_vx * ratio_threshold:
            dy = np.sign(vy)
        else:
            dx = np.sign(vx)
            dy = np.sign(vy)

        
        if 0 <= humans_grid_pos[i, 0] + dx < human_gridmap.shape[0] and 0 <= humans_grid_pos[i, 1] + dy < human_gridmap.shape[1]:
            human_gridmap[int(humans_grid_pos[i,0] + dx), int(humans_grid_pos[i,1] + dy)] = max(25, human_gridmap[int(humans_grid_pos[i,0] + dx), int(humans_grid_pos[i,1] + dy)])
            
            #print(f'x = {human[0]} y = {human[1]}, value = {human_gridmap[human[0], human[1]]}')

    return human_gridmap

place walls in map

In [61]:
def wall_in_map(map, x0, x1, y0, y1):
    Dx = x1 - x0
    Dy = y1 - y0

    n_of_samples = max(math.floor(max(abs(Dx), abs(Dy)) * 3000), 1)  # max to not divide by zero
    dy = Dy / n_of_samples
    dx = Dx / n_of_samples

    for step in range(n_of_samples + 1):
        x = math.floor(x0 + step * dx)
        y = math.floor(y0 + step * dy)
        # Ensure x and y are within the bounds of the map
        if 0 <= x < map.shape[0] and 0 <= y < map.shape[1]:
            map[x, y] = 255  # occupied

    return map

def walls_in_map(map, line_extremities):
    for i in range(len(line_extremities)):
        map = wall_in_map(map,*line_extremities[i])
    return map

Costmap output label generation. The label shall not include the real obstacles nor their inflation layer. It should only include the overall social cost that will then be stacked on top of all the other cost layers to build the complete costmap on top of which A* will run

In [62]:
# add queue obstacles in the label
def queue_cost(queue_label, q_obs):
    x_max = max(max(initial_state[:,0]), max(initial_state[:,4]), max(obs[:,1]), max(scaling_obs[:,1]))
    y_max = max(max(initial_state[:,1]), max(initial_state[:,5]), max(obs[:,3]), max(scaling_obs[:,3]))
    scale_obs2grid = np.diag([x_dim_grid/x_max, x_dim_grid/x_max, y_dim_grid/y_max, y_dim_grid/y_max])
    q_grid_obs = q_obs@scale_obs2grid
    queue_label = walls_in_map(queue_label, q_grid_obs)
    return queue_label

generate group in the middle of the simulation given a group diameter and number of people

In [63]:
def generate_group(radius, num_people):
    """
    Samples a given number of points uniformly within a circle of a given radius.
    """
    group_initial_state = [[]]
    for i in range(num_people):
        r = radius * np.sqrt(random.uniform(0.8, 1.2))
        theta = random.uniform(0, 2 * np.pi)
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        if i == 0: 
            group_initial_state[0] = [scaling_obs[0,0]/2 + x,scaling_obs[0,2]/2 + y, -x/(x**2 + y**2)**0.5, -y/(x**2 + y**2)**0.5, 0, 0]
        else: 
            group_initial_state.append([scaling_obs[0,0]/2 + x, scaling_obs[0,2]/2 + y, -x/(x**2 + y**2)**0.5, -y/(x**2 + y**2)**0.5, 0, 0])
   
    group_initial_state = np.array(group_initial_state)
    return group_initial_state

group costs

In [64]:
#add groups obstacles to the label
def group_cost(group_label, vertices):
    """
    Fill the polygon defined by vertices in the given image with the value 255.

    Parameters:
    - image (np.ndarray): The input image array.
    - vertices (np.ndarray): A numpy array of vertex coordinates.
                             Should be of shape (n, 2) where n is the number of vertices.

    Returns:
    - np.ndarray: The image with the polygon filled.
    """
    # Create a mask of the same size as the image, initialized to zeros
    mask = np.zeros_like(group_label, dtype=np.uint8)
    vertices = vertices[:, ::-1]
    vertices = vertices.reshape((-1, 1, 2))


    # Fill the polygon defined by vertices in the mask with 255
    cv2.fillPoly(mask, [vertices.astype(np.int32)], color=255)
    

    # Use the mask to change the corresponding area in the original image to 255
    group_label = np.maximum(mask, group_label)
    #group_label = np.where(mask == 255, 255, group_label)
    

    return group_label

people costs

In [65]:
# Function to update the costs in the costmap based on transformed people positions
#relevant part of the cpp code is from line 271  https://github.com/robotics-upo/nav2_social_costmap_plugin/blob/humble/src/social_layer.cpp
def people_costs(people_label, people):
    
    amplitude = 255.0
    covariance_front_height = 2.0
    covariance_front_width = 0.25
    covariance_rear_height = 0.25
    covariance_rear_width = 0.25
    covariance_right_height = 0.75
    covariance_right_width = 0.25
    covariance_when_still = 0.25
    use_passing = True
    resolution = 0.4# resolution assumed to be 40 cm per square
    tolerance_vel_still = 0.001#0.5
    cutoff = 10
    use_vel_factor = False
    speed_factor = 1

    # Calculate velocity magnitude and angle for each person
    for person in people:
        angle = np.arctan2(person[3], person[2])
        angle_right = angle - 1.57  # 90 degrees in radians
        cx = person[0] * resolution
        cy = person[1] * resolution

        # Iterate through each cell in the cost layer
        for i in range(x_dim_grid):
            for j in range(y_dim_grid):
                x = i * resolution
                y = j * resolution
                mag = (person[3]**2 + person[2]**2)**0.5
                if mag < tolerance_vel_still:
                    # PERSON STANDS STILL
                    a = calculate_gaussian(x, y, cx, cy, amplitude, covariance_when_still, covariance_when_still, 0)
                else:
                    ma = np.arctan2(y - cy, x - cx)
                    diff = shortest_angular_distance(angle, ma)


                    # FRONT
                    if math.fabs(diff) < math.pi / 2:
                        if use_vel_factor:
                            factor = 1.0 + mag * speed_factor
                            a = calculate_gaussian(x, y, cx, cy, amplitude, covariance_front_height * factor, covariance_front_width, angle)
                        else:
                            a = calculate_gaussian(x, y, cx, cy, amplitude, covariance_front_height, covariance_front_width, angle)
                    else:  # REAR
                        a = calculate_gaussian(x, y, cx, cy, amplitude, covariance_rear_height, covariance_rear_width, angle)

                    # RIGHT SIDE
                    if use_passing:
                        diff_right = shortest_angular_distance(angle_right, ma)
                        if math.fabs(diff_right) < math.pi / 2:
                            a_right = calculate_gaussian(x, y, cx, cy, amplitude, covariance_right_height, covariance_right_width, angle_right)
                            a = max(a, a_right)
                    
                    

                if a < cutoff:
                    continue

                # Update the cost in the costmap
                old_cost = people_label[i, j]
                people_label[i, j] = max(a, old_cost)
                
    return people_label


# Function to calculate Gaussian value
def calculate_gaussian(x, y, x0, y0, A, varx, vary, skew):
    dx = x - x0
    dy = y - y0
    h = np.sqrt(dx ** 2 + dy ** 2)
    angle = np.arctan2(dy, dx)
    mx = np.cos(angle - skew) * h
    my = np.sin(angle - skew) * h
    f1 = mx ** 2 / (2 * varx)
    f2 = my ** 2 / (2 * vary)
    return A * np.exp(-(f1 + f2))

rotate numpy vectors, the first two elements are rotated around (8,8), that's the center of the map

In [66]:
def rotate_vector(angle, vector):
    """
    Rotates the elements of the input vector around specified points.
    
    Parameters:
        angle (float): The angle by which to rotate the vector elements (in radians).
        vector (numpy array): A 1D numpy array with either 2, 4, or 6 elements.
        
    Returns:
        numpy array: The rotated vector.
    """
    if vector.size not in [2, 4, 6]:
        raise ValueError(f"Vector must have either 2, 4, or 6 elements.\nInstead it has {vector.size} elements")
    
    centers = [(8, 8), (0, 0), (8, 8)]
    cos_angle = np.cos(angle)
    sin_angle = np.sin(angle)
    
    rotated_vector = np.zeros_like(vector)
    
    for i in range(0, vector.size, 2):
        x, y = vector[i], vector[i+1]
        center = centers[i // 2]
        
        # Translate point to origin
        x_translated = x - center[0]
        y_translated = y - center[1]
        
        # Rotate point
        x_rotated = x_translated * cos_angle - y_translated * sin_angle
        y_rotated = x_translated * sin_angle + y_translated * cos_angle
        
        # Translate point back
        rotated_vector[i] = x_rotated + center[0]
        rotated_vector[i+1] = y_rotated + center[1]
    
    return rotated_vector

Here the gridmap that will be input for the neural network is obtained. Only the real obstacles are taken into account.

In [67]:
def save_gridmap_input(social_input, version):
    x_max = max(scaling_obs[:,1])
    y_max = max(scaling_obs[:,3])


    #define map and add obstacles
    #y_dim_grid = math.ceil(y_max/x_max *x_dim_grid)
    #grid_obs = np.empty((len(obs),2))
    #grid_obs
    # rescale obstacles coordinate for grid
    scale_obs2grid = np.diag([x_dim_grid/x_max, x_dim_grid/x_max, y_dim_grid/y_max, y_dim_grid/y_max])
    gridmap = np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8) # rows-> y

    # don't consider the queue obstacles, just the real ones
    if real_obs.size >0:
        grid_obs = real_obs@scale_obs2grid
        gridmap = walls_in_map(gridmap,grid_obs)
    gridmap = np.maximum(gridmap, social_input)

    #rescale robot start and goal to grid
    scale_robot2grid = np.diag([x_dim_grid/x_max,  y_dim_grid/y_max])
    robot_goal = tuple(np.uint8(np.floor(robot_goal_cont@scale_robot2grid)))
    
    map = instance.__file__.split("/")[-1].split(".")[0]
    parent_dir = os.path.abspath(os.getcwd())
    output_dir = os.path.join(parent_dir, 'images/in_out')
    os.makedirs(output_dir, exist_ok=True)
    cv2.imwrite(os.path.join(output_dir, f'{map}_step_{version}_in.jpg'), gridmap)
 





Here the label is saved

In [68]:
def save_costmap_output(social_cost_layer, version):
    map = instance.__file__.split("/")[-1].split(".")[0]
    parent_dir = os.path.abspath(os.getcwd())
    output_dir = os.path.join(parent_dir, 'images/in_out')
    os.makedirs(output_dir, exist_ok=True)
    cv2.imwrite(os.path.join(output_dir, f'{map}_step_{version}_out.jpg'), social_cost_layer)

create samples from all the instances available. Also apply rotations to the instances before feeding them into the input and output generation functions.

In [69]:
version = 0

#10 for queue, 100 for all groups, 0 for single person -> plus group script if generating person
for i in range(100):
    # instantiate all the necessary variables for the simulator
    obs = sim_init(instance)

    
    if people_in_group >0:
        additional_group = []
        group_initial_state = generate_group(group_radius, people_in_group)
        
        for i in range(people_in_group):
            additional_group.append(i)
        #print(additional_group)
        groups.append(additional_group)
        if initial_state.size >0:
            initial_state = np.append(group_initial_state, initial_state, axis=0)
        else: initial_state = group_initial_state
        
        




    
    # initiate the simulator
    s = psf.Simulator(
        initial_state,
        groups=groups,
        obstacles=obs, # don't instantiate queue obstacles in the sim
        config_file="/home/filippo-aisa/Projects/SocialEncDec/src/PySocialForce/examples/example.toml",
    )
    

    x_max =  max(scaling_obs[:,1])
    y_max = max(scaling_obs[:,3])


    queue_input =  np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    queue_label = np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    group_input =  np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    group_label = np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    non_group_input =  np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    non_group_label = np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    people_input = np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    people_label = np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    scale_hum2grid = np.diag([x_dim_grid/x_max,  y_dim_grid/y_max])

    lonely_humans = initial_state
    

    if queue_obs.size >0:
        q_on_grid = np.concatenate([queue_initial_state[:,:2]@scale_hum2grid, queue_initial_state[:,2:4]], axis=1)
        
        queue_input = humans_in_gridmap(queue_initial_state, queue_input)
        queue_input[int(np.floor(robot_goal_cont[0]*scale_hum2grid[0,0])), int(np.floor(robot_goal_cont[1]*scale_hum2grid[1,1]))] = 200
        
        # we don't add the people in the cost because we want A* to converge fast
        queue_label = queue_cost(queue_label, queue_obs)

        save_costmap_output(queue_label, version)
        save_gridmap_input(queue_input, version)
        version = version + 1
        save_costmap_output(queue_label.T, version)
        save_gridmap_input(queue_input.T, version)
        version = version + 1
        
        rotation_angles = np.random.uniform(0, 360, (30))
        for angle in rotation_angles:
            rotated_queue_input =  np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
            rotated_queue_label = np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
            rotated_q_initial_state = np.zeros_like(queue_initial_state)
            rotated_robot_goal = rotate_vector(angle, np.array(robot_goal_cont))
            for i in range(queue_initial_state.shape[0]):
                rotated_q_initial_state[i,:] = rotate_vector(angle, queue_initial_state[i,:])
                x_speed = rotated_robot_goal[0] - rotated_q_initial_state[i,0]
                y_speed = rotated_robot_goal[1] - rotated_q_initial_state[i,1]
                
                rotated_q_initial_state[i,2] = 0.1* x_speed + 0.1*random.uniform(-0.1 *x_speed, 0.1*x_speed)
                rotated_q_initial_state[i,3] = 0.1* y_speed + 0.1*random.uniform(-0.1 *y_speed, 0.1*y_speed)
            
            rotated_queue_input[int(np.floor(rotated_robot_goal[0]*scale_hum2grid[0,0])), int(np.floor(rotated_robot_goal[1]*scale_hum2grid[1,1]))] = 200
            q_on_grid = np.concatenate([rotated_q_initial_state[:,:2]@scale_hum2grid, rotated_q_initial_state[:,2:4]], axis=1)
            
            rotated_queue_input = humans_in_gridmap(rotated_q_initial_state, rotated_queue_input)
            
            rotated_q_obs = np.zeros_like(queue_obs)
            for i in range(queue_obs.shape[0]):
                rotated_q_obs[i,0:3:2] = rotate_vector(angle, queue_obs[i,0:3:2])
                rotated_q_obs[i,1::2] = rotate_vector(angle, queue_obs[i,1::2])
            # we don't add the people in the cost because we want A* to converge fast
            rotated_queue_label = queue_cost(rotated_queue_label, rotated_q_obs)



            save_costmap_output(rotated_queue_label, version)
            save_gridmap_input(rotated_queue_input, version)
            version = version + 1



    if len(groups) >0:
        for group in groups:
            if len(group) > 1:
                vertices = np.floor(initial_state[group,:2]@scale_hum2grid).astype(np.int32)
                group_label = group_cost(group_label, vertices)
                group_label = people_costs(group_label, np.concatenate([initial_state[group,:2]@scale_hum2grid, initial_state[group,2:4]], axis=1))
                group_input = humans_in_gridmap(initial_state[group],group_input)

                # if people are not facing the center of the group, then the group cost is not used, only the cost associated with the people is considered
                non_group_input = humans_in_gridmap(np.concatenate([initial_state[group,:2], initial_state[group,:2] - 8*np.ones_like(initial_state[group,:2])], axis=1),non_group_input)
                non_group_label = people_costs(non_group_label, np.concatenate([initial_state[group,:2]@scale_hum2grid, initial_state[group,:2] - 8*np.ones_like(initial_state[group,:2])], axis=1))
                map = instance.__file__.split("/")[-1].split(".")[0]
                parent_dir = os.path.abspath(os.getcwd())
                output_dir = os.path.join(parent_dir, 'images/in_out')
                os.makedirs(output_dir, exist_ok=True)
                cv2.imwrite(os.path.join(output_dir,f'non_{map}_step_{version}_out.jpg'), non_group_label)
                cv2.imwrite(os.path.join(output_dir,f'non_{map}_step_{version}_in.jpg'), non_group_input)
                
            
        
        people_in_group_indeces = [person for group in groups for person in group]
        mask = np.zeros(len(lonely_humans), dtype=bool)
        mask[people_in_group_indeces] = False
        lonely_humans = lonely_humans[mask]
    
    if lonely_humans.size >0:
        pass
        #TODO: here we are also considering the people in queue but we should not, 
        # also it seems like we are somehow seeing the robot start as a person
        #humans_in_gridmap(lonely_humans, people_input)
        #people_label = people_costs(people_label, lonely_humans)

    
    social_cost_layer = np.maximum(group_label, np.maximum(queue_label, people_label))
    social_input = np.maximum(queue_input, np.maximum(group_input, people_input))
    
    save_costmap_output(social_cost_layer, version)
    save_gridmap_input(social_input, version)
    version = version + 1


Create single human instances

In [70]:
""" version = 0
for i in range(1000):
    x = random.uniform(0.0, 16.0)
    y = random.uniform(0.0, 16.0)
    vx = random.uniform(-1,1)
    vy = random.uniform(-1,1)
    person = np.array([[x,y,vx,vy]])
    person_input =  np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    person_label = np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)
    person_input = humans_in_gridmap(person,person_input)
    person_label = people_costs(person_label, np.concatenate([person[:,:2]@scale_hum2grid, person[:,2:4]], axis=1))
    parent_dir = os.path.abspath(os.getcwd())
    output_dir = os.path.join(parent_dir, 'images/in_out')
    os.makedirs(output_dir, exist_ok=True)
    cv2.imwrite(os.path.join(output_dir, f'single_person_step_{version}_out.jpg'), person_label)
    cv2.imwrite(os.path.join(output_dir, f'single_person_step_{version}_in.jpg'), person_input)
    version = version+1 """

" version = 0\nfor i in range(1000):\n    x = random.uniform(0.0, 16.0)\n    y = random.uniform(0.0, 16.0)\n    vx = random.uniform(-1,1)\n    vy = random.uniform(-1,1)\n    person = np.array([[x,y,vx,vy]])\n    person_input =  np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)\n    person_label = np.zeros((x_dim_grid,y_dim_grid), dtype=np.uint8)\n    person_input = humans_in_gridmap(person,person_input)\n    person_label = people_costs(person_label, np.concatenate([person[:,:2]@scale_hum2grid, person[:,2:4]], axis=1))\n    parent_dir = os.path.abspath(os.getcwd())\n    output_dir = os.path.join(parent_dir, 'images/in_out')\n    os.makedirs(output_dir, exist_ok=True)\n    cv2.imwrite(os.path.join(output_dir, f'single_person_step_{version}_out.jpg'), person_label)\n    cv2.imwrite(os.path.join(output_dir, f'single_person_step_{version}_in.jpg'), person_input)\n    version = version+1 "